# Get historical market debt

* Query old blocks to get marketDebt for each market over time


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get the market debt

In [4]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)

def getMarketDebt(lookback):
    this_block = w3.eth.block_number

    if lookback == 0:
        block_lookback = this_block
    elif lookback >= 0:
        block_lookback = -1 * lookback
    else:
        raise ValueError("Lookback must be >= 0")
    
    block = w3.eth.get_block(this_block - lookback)
    
    marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call(block_identifier=block_lookback)

    markets = [{
        'asset': market[1].decode().replace('\x00', ''),
        'marketDebt': w3.fromWei(market[7], unit='ether'),
    } for market in marketSummaries]

    df_markets = pd.DataFrame(markets)
    df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
    df_markets = df_markets.set_index('asset').transpose()
    df_markets['block'] = block['number']
    df_markets['timestamp'] = block['timestamp']

    return df_markets



In [6]:
df = pd.DataFrame()
for i in range(1000):
    print(f'Running number {i}')
    df_this_block = getMarketDebt(i*100000)
    df = pd.concat([df, df_this_block], axis=0)

df

Running number 0
Running number 1
Running number 2
Running number 3
Running number 4
Running number 5
Running number 6
Running number 7
Running number 8
Running number 9
Running number 10
Running number 11
Running number 12
Running number 13
Running number 14
Running number 15
Running number 16
Running number 17
Running number 18
Running number 19
Running number 20
Running number 21
Running number 22
Running number 23
Running number 24
Running number 25
Running number 26
Running number 27
Running number 28
Running number 29
Running number 30
Running number 31
Running number 32
Running number 33
Running number 34
Running number 35
Running number 36
Running number 37
Running number 38
Running number 39
Running number 40
Running number 41
Running number 42
Running number 43
Running number 44
Running number 45
Running number 46
Running number 47
Running number 48
Running number 49
Running number 50
Running number 51
Running number 52
Running number 53
Running number 54
Running number 55
Ru

BadFunctionCallOutput: Could not decode contract function call to allProxiedMarketSummaries with return data: b'', output_types: ['(address,bytes32,bytes32,uint256,uint256,uint256,int256,uint256,int256,int256,(uint256,uint256,uint256,uint256,uint256,uint256,uint256))[]']

In [ ]:
assets = [c for c in df.columns if c not in ['block', 'timestamp']]
df[['block', 'timestamp'] + assets].sort_values('timestamp')